## Predicting Heart Disease


### Introduction:

Heart disease is an umbrella term for a vast range of conditions that affect the heart. While there are many leading causes that can contribute to higher risks to heart disease, unhealthy lifestyle choices and genetics often play a large role. In Canada, heart disease is the second leading cause of death and a leading cause of hospitalization (Government of Canada, 2022). This statistic leads us to wonder and study whether a predictive model can determine the presence of heart disease in a patient. Therefore, based on a data set from 1988 consisting of 4 databases from, Cleveland, Hungary, Switzerland, and Long Beach V, trends relating to heart disease will be used to predict the presence of the disease in individuals. 

This data was retrieved from the `heart_2.csv` file in the Heart Disease `project_data` folder, 
and was initially obtained from Kaggle: https://www.kaggle.com/datasets/johnsmith88/heart-disease-datasets


By observing the health reports of the patients in the data set, the five of the most influential factors in predicting the detection of heart disease in patients will be determined. From there, we will further investigate those variables further to determine the most significant of the five factors to include in a model to predict the presence of heart disease.

This will lead us to the question: **Can heart disease in a patient be predicted prior to diagnosis based upon the most influential factors within their health report?**

The goal of this project is to create a classification model that can accurately predict the presence of heart disease based on the most influential factors we determine. 

### Methods and Results:

Our general goal is to predict the `target` class by creating a K-NN Classification model with the optimal predictor variables that we will choose. Here are the steps that we will be taking to do this:


1. Read-in and tidy the data set: Make sure that the tidy data set criteria are all fulfilled.
2. Train/Test split: Split the data set into training and testing sets.
3. Analyze and summarize our training set: We will visualize the training data through the appropriate plots to see how the variables fluctuate and determine their relationship with the target variable.
4. Analyze predictor variables and select the ones of significance: We will create multiple recipes with different predictor variable combinations, and then analyze the resulting accuracy values from the cross-validations of each model to choose the most accurate one.
5. Find the optimal *k* value and build our final model.
6. Evaluation and Conclusion

### 1. Reading and Wrangling the Dataset
To begin our analysis we first loaded in the libraries that we would be using. Then, using `read_csv`, we first read in our dataset to better explore the variables and their relationship to the `target` variable, indicating the absence of heart disease, 0, and the presence of heart disease, 1. We then selected `age`, `exang`, `chol`, `thalach`, and `trestbps` as the variables we wanted to investigate. To check if there were missing values in the data set the `sum(is.na())` function was used.

In [ ]:
library(tidyverse)
library(repr)
library(dplyr)
library(tidymodels)
library(ggplot2)
library(gridExtra)
library(GGally)


url = 'https://raw.githubusercontent.com/jennywyzhao/dsci-group-project-002-14/main/project_data/heart.csv'
download.file(url, destfile = "project_data/heart_2.csv")
heart_data <- read_csv("project_data/heart_2.csv")

print("Table 1.1 Heart Data from Kaggle")
slice(heart_data, 1:5)

heart_var <- heart_data |>
    select(age, exang, chol, thalach, trestbps, target)

print("Table 2.1 Selected Variables for Investigation")

slice(heart_var, 1:5)
sum(is.na(heart_data))

### 2. Splitting the Data
From here, we split the data with the selected variables into a training set with 75% of the data, and a testing set with 25% of the data. To make sure the data is reproducible, we will be using 3456 as our seed number.

In [ ]:
set.seed(3456)

heart_split <- initial_split(heart_var, prop = 0.75, strata = target)  
heart_train <- training(heart_split)   
heart_test <- testing(heart_split)

### 3. Analyzing and Summarizing the Training Data

As our total dataset contains 1025 instances, and since our training set is 75% of that, there will be 768 instances in the training set.

In [ ]:
n_for_target <- heart_train |>
    group_by(target) |>
    summarize(n = n())

print("Table 3.1: Distribution of Diagnoses")
n_for_target

By using `group_by()` with num as its parameter, and `summarize()` with `n = n()` as its parameter, a tibble that contains the number of instances for each `target` type is created.

In `Table 3.1`, the ratio between the presence of heart disease (`target` = 1) vs. without heart disease (`target` = 0) is found to be nearly 1:1. 

In [ ]:
mean_per_diagnosis <- heart_train |>
    group_by(target) |>
    summarize(mean_age = mean(age), mean_exang = mean(exang), mean_chol = mean(chol), mean_thalach = mean(thalach), mean_trestbps = mean(trestbps))
print("Table 3.2: Mean of Variables Across Diagnoses")
mean_per_diagnosis

Again, `group_by()` and `summarize()` are used to group the data by their `target` diagnoses, and then to find the means of all the data within those subgroups.

* We can see that in Table 3.2, patients with a `target` = 0 are on average, only around 4 years older than patients who have `target` = 1.
* Angina induced by exercise (`exang`), seems to be in a negative relationship with the diagnoses.
* The maximum heart rate of a patient (`thalach`) is higher for patients without heart disease compared to patients who do. This could mean that thalach could help determine if a patient's `target` value is 1 (has heart disease). 
* It seems that many of the values across diagnoses seem to have similar or slightly different mean values, which may be a point of concern for our model.

#### Visualizing the predictor variables
To better understand the relationships between the variables, we chose to use box plots and stacked bar graphs. In this way, we can see the mean values for variables alongside the rang of values where there is a presence of heart disease, when `target` is 1, and when there is no heart disease, when the `target` is 0. Since the `target` variable only has two possible values, we decided to plot `target` on the x-axis.

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)
chol_plot <- ggplot(heart_train, 
     aes(y = chol, 
           x = as.factor(target))) +
     geom_boxplot(fill = "slateblue", alpha = 0.2)+
     labs(x = "Presence of Heart Disease (0 = no, 1 = yes)", y = "Cholesteral Level" )+
     theme(text = element_text(size = 20))+
     ggtitle("Serum Cholesterol Level vs. Presence of Heart Disease")

trestbps_plot<- ggplot(heart_train,
    aes(y = trestbps,
            x = as.factor(target))) +
    geom_boxplot(fill = "red", alpha = 0.2)+
    labs(x="Presence of Heart Disease (0 = no, 1 = yes)", y = "Resting Blood Pressure" )+
    theme(text = element_text(size = 20))+
    ggtitle("Resting Heart Blood Pressure vs. Presence of Heart Disease")

       
thalach_plot <- ggplot(heart_train,
        aes(y = thalach,
            x = as.factor(target))) +
    geom_boxplot(fill = "green", alpha = 0.2)+
    labs(x = "Presence of Heart Disease (0 = no, 1 = yes)", y = "Maximum Heart Rate Achieved" )+
    theme(text = element_text(size = 20))+
    ggtitle("Maximum Heart Rate Achieved vs. Presence of Heart Disease")

print("Figure 3.3: Variable Distributions Across Diagnosis")
grid.arrange(chol_plot, trestbps_plot, thalach_plot, ncol = 3)

age_plot<- ggplot(heart_train,
        aes(y = age,
            x = as.factor(target))) +
    geom_boxplot(fill = "purple", alpha = 0.2)+
    labs(x = "Presence of Heart Disease (0 = no, 1 = yes)", y = "Age" )+
    theme(text = element_text(size = 20))+
    ggtitle("Age vs. Presence of Heart Disease")

exang_plot <- ggplot(heart_train, aes(x = target, fill = as.factor(exang)))+
   geom_bar()+
   labs(x = "Presence of Heart Disease (0 = no, 1 = yes)", y = "Presence of Exercise Induced Angina")+
   theme(text = element_text(size = 20))+
   ggtitle("Exercise Induced Angina vs. Presence of Heart Disease")+
  scale_fill_discrete(name = "Presence of Angina (0 = no, 1 = yes)")

grid.arrange(age_plot, exang_plot, ncol = 2)

Looking at the above plot for serum cholesterol level, it can be seen the mean levels for individuals with heart disease is slightly lower than the mean cholesterol levels for individuals who do not have heart disease. This is slightly surprising as, the general trend found through research is that higher serum cholesterol levels tends to increase risks of developing heart disease. Since the difference is so slight, it does not make sense to include it as a predictor variable for our model.

Regarding resting blood pressure, there was not a large difference in the mean values between individuals with heart disease, and those without. From this graph, there is a suggestion that there is not a strong correlation between the presence of heart disease, and high or low resting blood pressure values. Hence, resting blood pressure is likely not a useful predictor variable for our model. In reference to the maximum heart rate achieved, there is a higher mean value for individuals with a presence of heart disease. This is supported by research which suggests that a higher resting heart rate could indicate an increased risk of developing heart disease. Thus, maximum heart rate achieved will likely be useful to include in our model.

Referencing the age plot and `Table 3.2`, the mean age value is smaller among the individuals who have heart disease. Since there is a difference between the mean ages of individuals who do and do not have heart disease, we will be using it as a predictor in our model. In general, the risks of heart disease increases as one ages, making this result somewhat unexpected. This result could be due to the fact that individuals who do not have heart disease tending to live longer or, sampling error.

Overall, we did not observe that the trends shown by the plot were similar to trends typically found and understood in other research.

#### The predictor variables that we will test are: `age`, `thalach`, and `exang`.

### 4. Analyzing and Selecting Predictor Variables

To confirm the selection of predictor variables, we will tune models and create distinct recipes for each set of predictor variables, then compare the accuracy of the models.
We will set neighbors to `tune()` in our K-NN recipe in combination with 5-fold cross-validation so that each model that we compare will be at its optimal accuracy.
Note that *k* values used for the cross-validation will be values from 1 to 50.
For now, it does not too significant that the accuracy is maximized (if we have the best *k* value), as we are analyzing how a predictor variable affects the model.
The data must also be scaled and centered, as the ranges of variables are different.

In [ ]:
heart_vfold <- vfold_cv(heart_train, v = 5, strata = target)

gridvals <- tibble(neighbors = seq(1, 50))

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

In [ ]:
h_recipe <- recipe(target ~ age, heart_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

h_results <- workflow() |>
    add_recipe(h_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = heart_vfold, grid = gridvals) |>
    collect_metrics() |>
    filter(.metric == "accuracy")

h_results_best <- h_results |>
    arrange(desc(mean)) |>
    slice(1) |>
    pull(mean)

In [ ]:
ht_recipe <- recipe (target ~ age + thalach, heart_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

ht_results <- workflow() |>
    add_recipe(ht_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = heart_vfold, grid = gridvals) |>
    collect_metrics() |>
    filter(.metric == "accuracy")

ht_results_best <- ht_results |>
    arrange(desc(mean)) |>
    slice(1) |>
    pull(mean)

In [ ]:
hto_recipe <- recipe (target ~ age + thalach + exang, data = heart_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

hto_results <- workflow() |>
    add_recipe(hto_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = heart_vfold, grid = gridvals) |>
    collect_metrics() |>
    filter(.metric == "accuracy")

hto_results_best <- hto_results |>
    arrange(desc(mean)) |>
    slice(1) |>
    pull(mean)

In [ ]:
model <- c("target ~ age", "target ~ age + thalach", "target ~ age + thalach + exang")
accuracies <- c(h_results_best, ht_results_best, hto_results_best)
acc_table <- tibble(`Recipe Input` = character(), Accuracy = numeric()) |>
    add_row(`Recipe Input` = model, Accuracy = accuracies)

print("Table 4.1: Resulting Accurracies of Different Predictor Variables")
acc_table

From `Table 4.1`, we can conclude that all three of these predictor variables are meaningful and increase the prediction accuracy. Using just `age`, the prediction accuracy is roughly 62% which, makes `age` a significant predictor variable. With the inclusion of `thalach` and `exang`, the accuracy increases significantly. Knowing this, we can now use the recipe with all three predictor variables to build our model. 

### 5. Finding the Optimal `k` Value and Building the Final Model

To begin, new components that need to be updated will be built for the final model. A new set of *k* values will need to be tested. We will thus tune our model with various *k* values, and pick the best one through a visualization. 

In [ ]:
k_values <- tibble(neighbors = seq(from = 1, to = 100, by = 1))

# heart_vfold, hto_recipe, and knn_spec are previously defined
patient_results <- workflow() |>
    add_recipe(hto_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = heart_vfold, grid = k_values) |>
    collect_metrics() |>
    filter(.metric == "accuracy")

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)

best_k_y <- patient_results |>
    filter(neighbors == 50) |>
    pull(mean)

k_accuracies_plot <- patient_results |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    geom_vline(xintercept = 50, colour = "steelblue") +
    geom_point(aes(x = 50, y = best_k_y), size = 3, colour = "blue") +
    labs(x = "K Value", y = "5 Average CV Accuracy") +
    ggtitle("Figure 5.1: Accuracies Across Different K Values") +
    theme(text = element_text(size = 20)) +
    theme(plot.title = element_text(hjust = 0.5))

k_accuracies_plot

We can see in `Figure 5.1` that at `k = 50`, there is a 3-point maximum plateau. This shows that there is a consistency in the maximum value, which increases the validity of this value. Knowing this, we will choose a value of *k* = 50 for our model.

#### Building the Final Model

Using our newly selected *k* value, we will build a new specification model as well as a new workflow.

In [ ]:
best_knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 80) |>
  set_engine("kknn") |>
  set_mode("classification")

# hto_recipe is previously defined
patient_fit <- workflow() |>
    add_recipe(hto_recipe) |>
    add_model(best_knn_spec) |>
    fit(data = heart_train)

print("Figure 5.2: Final Classification Model")
patient_fit

#### 6. Model Evaluation and Conclusion

Now, by fitting our testing data set (`heart_test`) with `patient fit`, we will evaluate the model that has been created. 

In [ ]:
patient_predictions <- predict(patient_fit, heart_test) |>
    bind_cols(heart_test)
patient_acc <- patient_predictions |>
    metrics(truth = target, estimate = .pred_class) |>
    select(.metric, .estimate) |>
    head(1)

print("Table 6.1: Model Accuracy on the Testing Set")
patient_acc

Here, we can see the final model's prediction accuracy is about 75.5% which, makes it decently reliable for prediction the heart disease in patients.

### Discussion:




In the end, we found the most significant predictors to be `age`, `thalach`, and `exang`, and the most optimal *k* value to be *k*=50. This resulting in the final models prediction accuracy to be approximately 75.5%. Since the final accuracy was pretty high , this classifier could be useful for predicting the presence of heart disease. With that in mind, it is important to remember that when put to use, the values for age, maximum heart rate achieved, and exercise induced angina may be less consistent with the values used to train and test this model. This may result to more frequent instances of missclassification, and a slightly different accuracy in practice.

Overall, we were not expecting age to be as prominent of a factor as it was in making an accurate model. During the initial researching stage, we found that as age increases, the risk for heart disease also increases as, the heart and its connecting blood vessels are also aging (NIH, 2023). This however was not the trend we found when looking at the mean values for individuals who do have heart disease which, was a lower age compared to individuals without heart disease. This finding may lead to researchers looking into a younger age group when trying to find ways to identify risks for developing heart disease. 

The maximum heart rate achieved was also found to be a significant factor in detecting presence of heart disease. This brings to light a new how going over that heart rate can also increases risks of heart disease. As supported by Healthline, going over the maximum heart rate for an individuals age group can be dangerous for ones health (Healthline, 2023). This particular finding may highlight the importance of the maximum heart rate for individuals, and what a healthy and safe heart rate range is to reduce risks of developing heart disease.

Exercise induced angina was not found in high instances in patients with heart disease. This was quite surprising but, made it an effective predictor variable as, it was very infrequently found  when an individual had heart disease. While this finding is unexpected, it highlights the significance of the risks of heart disease, even in the absence of some of the telltale signs.

For the future, these findings may lead to different directions in research where the less expected predictors and parts of a health report can be used to determine risks of heart disease before a diagnosis is made. In turn, this may lead to questions of whether the current direction of research is the most effective, and if the stereotypical of a heart disease patient still holds true.

### References:


Canada, P. H. A. of. (2022, July 28). Government of Canada. Canada.ca. Retrieved April 9, 2023, from https://www.canada.ca/en/public-health/services/publications/diseases-conditions/heart-disease-canada.html 

Chertoff, J. (2023, February 23). Running heart rate: What's safe and what's too high? Healthline. Retrieved April 19, 2023, from https://www.healthline.com/health/running-heart-rate#:~:text=Going%20higher%20than%20your%20maximum,you're%20new%20to%20exercise. 

Lapp, D. (2019, June 6). Heart disease dataset. Kaggle. Retrieved April 12, 2023, from https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset?resource=download 

Mayo Foundation for Medical Education and Research. (2022, March 30). Angina. Mayo Clinic. Retrieved April 9, 2023, from https://www.mayoclinic.org/diseases-conditions/angina/symptoms-causes/syc-20369373#:~:text=During%20times%20of%20low%20oxygen,when%20exercising%2C%20angina%20can%20result. 

U.S. Department of Health and Human Services. (2018, June 1). Heart health and aging. National Institute on Aging. Retrieved April 12, 2023, from https://www.nia.nih.gov/health/heart-health-and-aging#:~:text=Adults%20age%2065%20and%20older,risk%20of%20developing%20cardiovascular%20disease. 